In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

from ib_insync import util, IB
util.startLoop()

# Set the root
from from_root import from_root
ROOT = from_root()

# set vscode path
import sys
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>

In [ ]:
from src.nse import assemble_nse_underlyings, Vars
import asyncio

MARKET = 'NSE'

_vars = Vars(MARKET)

In [ ]:
with await IB().connectAsync(port=_vars.PORT) as ib:
    output = asyncio.run(assemble_nse_underlyings(ib))


In [ ]:
output

In [ ]:
from src.utils import get_pickle


get_pickle(ROOT/'data'/'nse'/'unds.pkl')